#### DNN기반 다중분류 모델 구현
 - 데이터셋 : iris.csv
 - feature : 4개 Sepal_Length, Sepal_Width, Petal_Length, Petal_Width
 - target/label : 1개 variety
 - 학습방법 : 지도학습 > 분류 > 다중분류 (클래스 3개)
 - 알고리즘 : 인공신경망(ANN) -> MLP, DNN : 은닉층이 많은 구성
 - 프레임워크 : Pytorch

[1] 모듈 로딩 및 데이터 준비
***

In [9]:
# - 모델 관련
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torchmetrics.classification import MulticlassF1Score
from torchinfo import summary

# - 데이터 관련
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split

In [10]:
# 활용 패키지 버전 체크  ==> 사용자 정의 함수로 구현해놓기
def checkversion() :
    print(f'Pytorch v {torch.__version__}')
    print(f'pandas v {pd.__version__}')

In [11]:
#### 데이터 로딩
irisDF = pd.read_csv(r'C:\Users\hoon\Desktop\경대 KDT 6기\EX_PANDAS6-main\EX_PANDAS6-main\TORCH_DL\data\iris.csv')
irisDF.head(2)

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa


In [12]:
### 타겟 변경 ==> 정수화, 클래스3개 => 2개
labels = dict(zip(irisDF['variety'].unique().tolist(),range(3)))
print(f'labels => {labels}')

irisDF['variety'] = irisDF['variety'].replace(labels)

labels => {'Setosa': 0, 'Versicolor': 1, 'Virginica': 2}


[2] 모델 클래스 설계 및 정의<hr>
 - 클래스 목적 : iris 데이터 학습 및 추론
 - 클래스 이름 : IrisMCFModel
 - 부모 클래스 : nn.Module
 - 매개변수    : 층별 입/출력 개수 고정 => 매개변수 필요 없음 
 - 속성/필드   :  
 - 클래스 기능 : _ _ init _ _() : 모델 구조 설정, forward() : 순방향 학습  <= 오버라이딩(overriding)
 - 클래스 구조 
   * 입력층 : 입력 4개     출력 10개 (퍼셉트론 10개 존재)
   * 은닉층 : 입력 10개    출력 5개  (퍼셉트론 5개 존재)
   * 출력층 : 입력 5개     출력 1개 (다중분류)

 - 활성화함수
   * 클래스 형태 ==> nn.MELoss, nn.ReLU  -> _ _ init _ _() 메서드에서 사용
   * 함수 형태  ==> torch.nn.functional 아래에 -> forward() 메서드에서 사용

In [13]:
class irisMCFModel(nn.Module) :

    # 모델 구조 구성 및 인스턴스 생성 메서드
    def __init__(self) :
        super().__init__()
        self.in_layer = nn.Linear(4,10)
        self.h_layer = nn.Linear(10,5)
        self.out_layer = nn.Linear(5,3)     # 다중분류 'Setosa', 'Ver', 'Vir'

    # 순방향 학습 진행 메서드
    def forward(self, data) :
        # 입력층
        y = self.in_layer(data)             # f1w1+f2w2+f3w3+...+f10w10+b
        F.relu(y)                           # relu => y 값의 범위 : 0 < y
        
        # 은닉층 : 10개의 숫자값이 들어옴(>=0)
        y = self.h_layer(y)
        F.relu(y)

        # 출력층 : 5개의 숫자 값 => 다중분류 : 손실함수 crossEntropyLoss 내부에서 Softmax 처리 해서 pytorch에서는 안넣어도 됨
        return self.out_layer(y)

In [14]:
### 모델 인스턴스 생성
model = irisMCFModel()
print(model)

irisMCFModel(
  (in_layer): Linear(in_features=4, out_features=10, bias=True)
  (h_layer): Linear(in_features=10, out_features=5, bias=True)
  (out_layer): Linear(in_features=5, out_features=3, bias=True)
)


In [15]:
### [테스트] 모델 사용 메모리 정보 확인
summary(model, input_size=(10,4))

Layer (type:depth-idx)                   Output Shape              Param #
irisMCFModel                             [10, 3]                   --
├─Linear: 1-1                            [10, 10]                  50
├─Linear: 1-2                            [10, 5]                   55
├─Linear: 1-3                            [10, 3]                   18
Total params: 123
Trainable params: 123
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

[3] 데이터셋 클래스 설계 및 정의 <hr>

In [16]:
class IrisDataset(Dataset) :

    def __init__(self,featureDF, targetDF) :
        self.featureDF = featureDF
        self.targetDF = targetDF
        self.n_rows = featureDF.shape[0]
        self.n_features = featureDF.shape[1]

    def __len__(self) :
        return self.n_rows

    def __getitem__(self, index) :
        
        # 텐서화
        featureTS = torch.FloatTensor(self.featureDF.iloc[index].values)
        targetTS = torch.FloatTensor(self.targetDF.iloc[index].values)

        # 피쳐와 타겟 반환
        return featureTS, targetTS

In [17]:
## [테스트] 데이터셋 인스턴스 생성

# - DataFrame에서 피쳐와 타겟 추출
featureDF = irisDF[irisDF.columns[:-1]]
targetDF = irisDF[irisDF.columns[-1:]]


# - 커스텀 데이터셋 인스턴스 생성
irisDS = IrisDataset(featureDF,targetDF)

# - 데이터로더 인스턴스 생성        <= DataLoader로 테스트 해야함 for문으로 feature랑 label 하나씩 뽑아서 확인하믄 됨
irisDL = DataLoader(irisDS)
for feature, label in irisDL :
    print(feature.shape, label.shape)
    break

torch.Size([1, 4]) torch.Size([1, 1])


[4] 학습 준비 <hr>
 - 학습횟수 : EPOCH          <- 처음 ~ 끝까지 공부하는 단위  
 - 배치크기 : BATCH_SIZE     <- 한번에 학습할 데이터 양
 - 위치지정 : DEVICE         <- 텐서 저장 및 실행 위치 (GPU/CPU)
 - 학습률(lr) : 가중치와 절편 업데이트 시 경사하강법으로 업데이트 간격 설정 (0.001 ~ 0.1) <- 하이퍼파라미터

In [18]:
### 학습 진행 관련 설정값
EPOCH = 1000
BATCH_SIZE = 10
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LR = 0.001

 - 학습 준비에 필요한 것
    * 인스턴스/객체 : 모델, 데이터셋, 최적화, (, 손실함수, 성능지표)

In [19]:
# 찐 모델 인스턴스 생성
model = irisMCFModel()

# - 학습용, 검증용, 테스트용 데이터셋 분리
X_train, X_test, y_train, y_test = train_test_split(featureDF,targetDF,random_state = 1)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,random_state = 1)

# - 학습용, 검증용, 테스트용 인스턴스 생성
trainDS = IrisDataset(X_train,y_train)
valDS = IrisDataset(X_val,y_val)
testDS = IrisDataset(X_test, y_test)

# - 학습용 데이터로더 인스턴스 생성
trainDL = DataLoader(trainDS,batch_size=BATCH_SIZE)

# 최적화 인스턴스 - 최소의 손실을 만드는 w,b 찾는 작업 해주는 인스턴스
# 최적화 인스턴스에 model.parameters() 전달 필요
optimizer = optim.Adam(model.parameters(),lr=LR)


# 손실함수 인스턴스 > 분류 > 다중분류 CrossEntropyLoss  
#                                  예측값은 선형식 결과값으로 전달 
crossLoss = nn.CrossEntropyLoss()

[5] 학습 진행<hr>

In [20]:
## 학습의 효과 확인을 위해 손실값과 성능평가값 저장 필요
LOSS_HISTORY, SCORE_HISTORY = [[],[]],[[],[]]
CNT = len(trainDL)

for epoch in range(EPOCH) :
    # 학습 모드로 모델 설정
    model.train()


    # 배치크기만큼 데이터 로딩해서 학습 진행
    loss_total , score_total = 0, 0
    for featureTS, targetTS in trainDL :
        
        # 학습 진행
        pre_y = model(featureTS)

        # 손실 계산 : nn.CrossEntropyLoss 요구사항 : 정답/타겟은 0D 또는 1D, 타입은 long
        loss = crossLoss(pre_y,targetTS.reshape(-1).long())
        loss_total += loss.item()

        # 성능 평가 계산
        score = MulticlassF1Score(num_classes=3)(pre_y,targetTS.reshape(-1))
        score_total += score.item()

        # 최적화 진행
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # 에포크 당 검증기능
    # 모델 검증 모드 설정
    model.eval()
    with torch.no_grad():
        # 검증 데이터셋
        val_featureTS = torch.FloatTensor(valDS.featureDF.values)
        val_targetTS = torch.FloatTensor(valDS.targetDF.values)
        # 추론/평가
        pre_val = model(val_featureTS)
        # 손실계산
        loss_val = crossLoss(pre_val, val_targetTS.reshape(-1).long())
        # 성능평가
        score_val = MulticlassF1Score(num_classes=3)(pre_val, val_targetTS.reshape(-1))



    # 손실값과 성능평가값 저장
    LOSS_HISTORY[0].append(loss_total/CNT) 
    SCORE_HISTORY[0].append(score_total/CNT)
    LOSS_HISTORY[1].append(loss_val) 
    SCORE_HISTORY[1].append(score_val)  

In [23]:
print(LOSS_HISTORY[0])

[1.0276528663105435, 0.9974947306844923, 0.9820403655370077, 0.9713784721162584, 0.95998082558314, 0.947044226858351, 0.9329726894696554, 0.9179114103317261, 0.9018204874462552, 0.8846118582619561, 0.8661623332235548, 0.8463511334525214, 0.8251092963748508, 0.8024374975098504, 0.77840123573939, 0.7531267868147956, 0.7268027597003512, 0.6996843616167704, 0.6720912125375536, 0.644393159283532, 0.6169817712571886, 0.5902340412139893, 0.5644751257366605, 0.5399504039022658, 0.516812351014879, 0.4951232969760895, 0.4748704830805461, 0.4559868375460307, 0.43837259544266594, 0.4219125476148393, 0.4064892960919274, 0.3919908040099674, 0.37831513418091667, 0.3653721743159824, 0.35308431254492867, 0.3413859291209115, 0.3302227192454868, 0.3195501251353158, 0.30933252970377606, 0.29954150319099426, 0.2901545928584205, 0.2811541226175096, 0.2725258635150062, 0.2642581644985411, 0.2563411709335115, 0.2487659901380539, 0.24152426587210762, 0.2346077263355255, 0.22800799873140123, 0.22171640147765478